In [2]:
import pandas as pd
from pandas import read_excel
import numpy as np
import matplotlib.pylab as plt
import matplotlib.pyplot
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from string import ascii_letters
import seaborn as sns
from datetime import datetime
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

In [3]:
data = read_excel("monthly_data.xlsx")
dfa = pd.read_excel (r'monthly_data.xlsx')

data['year']=pd.to_datetime(data['year'], format = '%Y-%m')
data.set_index('year', inplace=True)
cols = data.columns
train_size = int(len(data) * 0.8)
df_train, df_test = data[0:train_size], data[train_size:]

print(len(df_train))
print(len(df_test))
print(data.head())

48
12
            max_temp  min_temp  rainfall(mm)  humidity  dengue  Leptospirosis
year                                                                         
2014-01-01      32.8      22.2         0.000     67.75       1              0
2014-02-01      32.8      22.8         7.650     71.25       0              0
2014-03-01      34.1      24.2        27.670     72.37       0              0
2014-04-01      34.3      25.7       100.375     77.25       5              0
2014-05-01      33.9      25.3       290.120     79.12      28              0


In [4]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
#since the test works for only 12 variables, I have randomly dropped
#in the next iteration, I would drop another and check the eigenvalues
johan_test_temp = data.drop([ 'max_temp'], axis=1)
coint_johansen(johan_test_temp,-1,1).eig

array([5.95020960e-01, 4.67640257e-01, 3.63565519e-01, 2.32856432e-01,
       2.56061417e-06])

In [5]:
from statsmodels.tsa.vector_ar.var_model import VAR
model = VAR(endog=df_train)
model_fit = model.fit()

C:\Users\Public\anaconda3\envs\machine_learning\lib\site-packages\statsmodels\tsa\base\tsa_model.py:159: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


In [6]:
# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(df_test))

In [7]:
#make final predictions
model = VAR(endog=data)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=12)
print(yhat)

[[ 3.22266251e+01  2.38367190e+01  1.58869888e+01  7.43696465e+01
   4.45376557e+00 -1.21607303e+00]
 [ 3.26583136e+01  2.43668223e+01  2.18941295e+01  7.37491441e+01
  -2.10028422e+00 -2.23194208e+00]
 [ 3.28488976e+01  2.47212382e+01  6.63810793e+01  7.44592951e+01
   1.15302567e+00 -2.10877451e+00]
 [ 3.28003252e+01  2.48897832e+01  1.31846643e+02  7.58629121e+01
   1.05167908e+01 -2.39695007e-01]
 [ 3.25969512e+01  2.49242097e+01  1.97515096e+02  7.74223185e+01
   2.11683960e+01  2.68995284e+00]
 [ 3.23348843e+01  2.48807281e+01  2.49745599e+02  7.87751145e+01
   3.00350954e+01  5.69402926e+00]
 [ 3.20866709e+01  2.48027363e+01  2.83310041e+02  7.97404880e+01
   3.58739256e+01  8.08447102e+00]
 [ 3.18935317e+01  2.47195865e+01  2.98984868e+02  8.02825723e+01
   3.86688523e+01  9.58637351e+00]
 [ 3.17699642e+01  2.46488984e+01  3.00744396e+02  8.04597022e+01
   3.90414500e+01  1.02331439e+01]
 [ 3.17119462e+01  2.45989899e+01  2.93619127e+02  8.03764048e+01
   3.78369390e+01  1.0222

C:\Users\Public\anaconda3\envs\machine_learning\lib\site-packages\statsmodels\tsa\base\tsa_model.py:159: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


In [8]:
yhat[yhat<0] = 0
new_arr1 = np.ceil(yhat)
new_arr1

array([[ 33.,  24.,  16.,  75.,   5.,   0.],
       [ 33.,  25.,  22.,  74.,   0.,   0.],
       [ 33.,  25.,  67.,  75.,   2.,   0.],
       [ 33.,  25., 132.,  76.,  11.,   0.],
       [ 33.,  25., 198.,  78.,  22.,   3.],
       [ 33.,  25., 250.,  79.,  31.,   6.],
       [ 33.,  25., 284.,  80.,  36.,   9.],
       [ 32.,  25., 299.,  81.,  39.,  10.],
       [ 32.,  25., 301.,  81.,  40.,  11.],
       [ 32.,  25., 294.,  81.,  38.,  11.],
       [ 32.,  25., 283.,  81.,  36.,  10.],
       [ 32.,  25., 271.,  80.,  34.,  10.]])

In [9]:
COLUMN_NAMES=['year','max_temp','min_temp','rainfall','humidity','dengue','Leptospirosis']
mvts_df = pd.DataFrame(columns=COLUMN_NAMES)

In [10]:
psudo_df=pd.DataFrame(new_arr1)
psudo_df.head()

,0,1,2,3,4,5
0,33.0,24.0,16.0,75.0,5.0,0.0
1,33.0,25.0,22.0,74.0,0.0,0.0
2,33.0,25.0,67.0,75.0,2.0,0.0
3,33.0,25.0,132.0,76.0,11.0,0.0
4,33.0,25.0,198.0,78.0,22.0,3.0


In [11]:
list(psudo_df.columns)

[0, 1, 2, 3, 4, 5]

In [12]:
d = pd.date_range(start='1/1/2019', end='12/1/2019', freq='MS')

In [13]:
mvts_df['year']=d
mvts_df['max_temp']=psudo_df[0]
mvts_df['min_temp']=psudo_df[1]
mvts_df['rainfall']=psudo_df[2]
mvts_df['humidity']=psudo_df[3]
mvts_df['dengue']=psudo_df[4]
mvts_df['Leptospirosis']=psudo_df[5]

In [14]:
mvts_df

,year,max_temp,min_temp,rainfall,humidity,dengue,Leptospirosis
0,2019-01-01,33.0,24.0,16.0,75.0,5.0,0.0
1,2019-02-01,33.0,25.0,22.0,74.0,0.0,0.0
2,2019-03-01,33.0,25.0,67.0,75.0,2.0,0.0
3,2019-04-01,33.0,25.0,132.0,76.0,11.0,0.0
4,2019-05-01,33.0,25.0,198.0,78.0,22.0,3.0
5,2019-06-01,33.0,25.0,250.0,79.0,31.0,6.0
6,2019-07-01,33.0,25.0,284.0,80.0,36.0,9.0
7,2019-08-01,32.0,25.0,299.0,81.0,39.0,10.0
8,2019-09-01,32.0,25.0,301.0,81.0,40.0,11.0
9,2019-10-01,32.0,25.0,294.0,81.0,38.0,11.0
